<a href="https://colab.research.google.com/github/FranciscoJSSantos/Analise_dos_dados_criminais_brasil/blob/main/Analise_dos_dados_criminais_brasil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de Dados Criminais

O objetivo dessa tarefa é construir um mapa do crime no Brasil a partir de dados abertos. O trabalho pode ser feito em dupla ou individual. Cada dupla ou pessoa deve fazer uma análise e pré-processamento dos dados para que possam construir um mapa do crime do Brasil. Além disso, vocês devem aplicar algoritmos de de clusterização para agrupar cidades de acordo com o nível de criminalidade (como o exemplo dos Estados Unidos que apresentei em sala de aula).

In [9]:
# Imports necessários
import pandas as pd
from sklearn import preprocessing
from scipy.spatial import distance_matrix
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
import scipy.cluster.hierarchy as shc
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
import plotly.graph_objects as go

In [10]:
# Carregado base ocorrencia
data_ocorrenciasOcorrencia = pd.read_excel("/content/drive/MyDrive/trabalho pratico 2/indicadoressegurancapublicauf.xlsx",sheet_name='Ocorrências', names=['UF', 'Tipo Crime', 'Ano', 'Mês', 'Ocorrências'])
data_ocorrenciasOcorrencia.head(5)

,UF,Tipo Crime,Ano,Mês,Ocorrências
0,Acre,Estupro,2021,janeiro,39
1,Acre,Furto de veículo,2021,janeiro,55
2,Acre,Homicídio doloso,2021,janeiro,13
3,Acre,Lesão corporal seguida de morte,2021,janeiro,0
4,Acre,Roubo a instituição financeira,2021,janeiro,0


In [11]:
# Carregado base vitima
data_ocorrenciasVitima = pd.read_excel("/content/drive/MyDrive/trabalho pratico 2/indicadoressegurancapublicauf.xlsx",sheet_name='Vítimas', names=['UF','Tipo Crime', 'Ano', 'Mês', 'Sexo', 'Vítimas'])
data_ocorrenciasVitima.head(5)

,UF,Tipo Crime,Ano,Mês,Sexo,Vítimas
0,Acre,Homicídio doloso,2021,janeiro,Feminino,1
1,Acre,Homicídio doloso,2021,janeiro,Masculino,12
2,Acre,Homicídio doloso,2021,janeiro,Sexo NI,0
3,Acre,Homicídio doloso,2021,fevereiro,Feminino,4
4,Acre,Homicídio doloso,2021,fevereiro,Masculino,12


In [13]:
##juntando as duas bases
data_ocorrenciasOcorrencia['UF'] = data_ocorrenciasVitima['UF']
data_ocorrenciasVitima.head(20)



,UF,Tipo Crime,Ano,Mês,Sexo,Vítimas
0,Acre,Homicídio doloso,2021,janeiro,Feminino,1
1,Acre,Homicídio doloso,2021,janeiro,Masculino,12
2,Acre,Homicídio doloso,2021,janeiro,Sexo NI,0
3,Acre,Homicídio doloso,2021,fevereiro,Feminino,4
4,Acre,Homicídio doloso,2021,fevereiro,Masculino,12
5,Acre,Homicídio doloso,2021,fevereiro,Sexo NI,0
6,Acre,Homicídio doloso,2021,março,Feminino,2
7,Acre,Homicídio doloso,2021,março,Masculino,24
8,Acre,Homicídio doloso,2021,março,Sexo NI,0
9,Acre,Homicídio doloso,2021,abril,Feminino,2
